### Notebook to generate Training + Testing Sets for campaign 1 data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from fuchs_model import *

In [2]:
seed = 2
tag = 'fuchs_v5_0_seed-' + str(seed)

num_testing_points = 250000 # Number of Points in Testing Dataset
pct_noise = 10 # Percentage of Noise in Dataset
percentPoints = 100 # Percentage of Points to use from Training Set

training_tag = tag + "_train_campaign2_pct_" + str(percentPoints) + "_noise_" + str(pct_noise)
testing_tag = tag + "_test_campaign2_points_" + str(num_testing_points)

In [3]:
df = pd.read_csv('datasets/Campaign2.csv') # table of input parameter combinations
num_training_points = int(df.shape[0] * percentPoints/100 + 0.5)
df = df.sample(frac=percentPoints/100, random_state=seed)

In [4]:
# Free parameters from Fuchs model
c1 = 25
c2 = 0.74
theta = 25
const_f = False

In [5]:
intensity_min = 22 # In W/m^2
intensity_max = 23
focal_distance_min = 0e-6 # In m
focal_distance_max = 0.0e-6
target_thickness_min = 0.5e-6 # In m
target_thickness_max = 5.0e-6
# FWHM Period and Spot Size Constant
t_FWHM_min = 40.0e-15
t_FWHM_max = 40.0e-15
spot_size_min = 1.5e-6
spot_size_max = 1.5e-6
contrast_min = -7
contrast_max = -6
t_delay = 0.06e-9 # 60 picosecond delay between pre-pulse and main pulse

In [6]:
df_array = df.to_numpy(dtype=np.float64)
intensities = df_array[:, 2] # Intensity in 3rd column of array
intensities *= 1e4
focal_distances = np.zeros(num_training_points) # Focal Depth constant for all points in campaign 2
focal_distances *= 1e-6
thicknesses = df_array[:, 0] # Target Thickness in 1st column of array
thicknesses *= 1e-6
contrasts = df_array[:, 1] # Contrast in 2nd column of array
time_FWHMs = np.ones(num_training_points) * 40.0e-15 # FWHM Period constant for all points in campaign 1
spot_sizes = np.ones(num_training_points) * 1.5e-6 # Spot Size constant for all points in campaign 1

In [7]:
tmp = fuchs_function_with_prepulse(intensities, focal_distances, thicknesses,contrast=contrasts,t0=t_delay, pct_noise=pct_noise, spectrum=False)
noisy_max_proton_energy = tmp[0]
noisy_total_proton_energy = tmp[1]
noisy_average_proton_energy = tmp[2]
efficiency = tmp[3]
laser_energy = tmp[4]
max_proton_energy = tmp[5]
total_proton_energy = tmp[6]
average_proton_energy = tmp[7]

df = pd.DataFrame({'Intensity': intensities*1e-4, 'Pulse Duration': time_FWHMs*1e15, 'Target Thickness': thicknesses*1e6, 'Spot Size': spot_sizes*1e6, 'Focal Distance': focal_distances*1e6, 'Contrast': contrasts, 'Max Proton Energy': noisy_max_proton_energy,
                    'Total Proton Energy': noisy_total_proton_energy, 'Avg Proton Energy': noisy_average_proton_energy, 'Max Exact Energy': max_proton_energy, 'Total Exact Energy': total_proton_energy, 'Avg Exact Energy': average_proton_energy, 'Laser Energy': laser_energy * 1e3, 'Conversion Efficiency': efficiency})
df

Shape (1274091, 10)


,Intensity,Pulse Duration,Target Thickness,Spot Size,Focal Distance,Contrast,Max Proton Energy,Total Proton Energy,Avg Proton Energy,Max Exact Energy,Total Exact Energy,Avg Exact Energy,Laser Energy,Conversion Efficiency
0,2.075210e+18,40.0,2.00,1.5,0.0,8.408530e-07,0.038206,1.735314e+06,0.010932,0.040027,1.817997e+06,0.011453,1.593884,0.000183
1,5.540000e+18,40.0,1.75,1.5,0.0,4.928960e-07,0.065601,7.659267e+06,0.018493,0.066299,7.740793e+06,0.018690,3.357397,0.000369
2,4.982550e+18,40.0,0.90,1.5,0.0,3.863900e-07,0.137420,3.051470e+07,0.034538,0.170905,3.795010e+07,0.042953,3.664808,0.001659
3,5.487910e+18,40.0,4.05,1.5,0.0,2.482800e-07,0.263932,8.326822e+07,0.063755,0.225213,7.105278e+07,0.054402,4.627281,0.002460
4,6.545080e+18,40.0,4.20,1.5,0.0,7.614780e-07,0.008320,4.134328e+04,0.008320,0.004661,4.968895e+04,0.001490,2.111010,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274086,9.806310e+18,40.0,4.40,1.5,0.0,3.225790e-07,0.079854,1.444482e+07,0.022734,0.071472,1.292851e+07,0.020347,5.338473,0.000388
1274087,2.046970e+18,40.0,3.90,1.5,0.0,3.730460e-07,0.119606,1.299483e+07,0.030383,0.118255,1.284808e+07,0.030040,2.019537,0.001019
1274088,6.378190e+18,40.0,4.50,1.5,0.0,1.567050e-07,0.388862,2.041422e+08,0.083051,0.429113,2.252726e+08,0.091647,5.900225,0.006116
1274089,4.668630e+18,40.0,1.20,1.5,0.0,5.573140e-07,0.058140,5.876237e+06,0.016369,0.063555,6.423486e+06,0.017893,2.919292,0.000352


In [8]:
df.to_hdf('datasets/' + training_tag + '.h5', key='df', mode='w')

In [9]:
### Testing set generation
np.random.seed(seed)
intensities = 10**np.random.uniform(intensity_min, intensity_max, num_testing_points)
focal_distances = np.zeros(num_testing_points) # Focal Depth constant for all points in campaign 2
thicknesses = np.random.uniform(target_thickness_min, target_thickness_max, num_testing_points)
contrasts = 10**np.random.uniform(contrast_min, contrast_max, num_testing_points)
time_FWHMs = np.ones(num_testing_points) * 40.0e-15 # FWHM Period constant for all points in campaign 2
spot_sizes = np.ones(num_testing_points) * 1.5e-6 # Spot Size constant for all points in campaign 2

In [10]:
tmp = fuchs_function_with_prepulse(intensities, focal_distances, thicknesses,contrast=contrasts,t0=t_delay, pct_noise=0, spectrum=False)
noisy_max_proton_energy = tmp[0]
noisy_total_proton_energy = tmp[1]
noisy_average_proton_energy = tmp[2]
efficiency = tmp[3]
laser_energy = tmp[4]
max_proton_energy = tmp[5]
total_proton_energy = tmp[6]
average_proton_energy = tmp[7]
test_df = pd.DataFrame({'Intensity': intensities*1e-4, 'Pulse Duration': time_FWHMs*1e15, 'Target Thickness': thicknesses*1e6, 'Spot Size': spot_sizes*1e6, 'Focal Distance': focal_distances*1e6, 'Contrast': contrasts, 'Max Proton Energy': noisy_max_proton_energy,
                    'Total Proton Energy': noisy_total_proton_energy, 'Avg Proton Energy': noisy_average_proton_energy, 'Max Exact Energy': max_proton_energy, 'Total Exact Energy': total_proton_energy, 'Avg Exact Energy': average_proton_energy, 'Laser Energy': laser_energy * 1e3, 'Conversion Efficiency': efficiency})
test_df

Shape (250000, 10)


,Intensity,Pulse Duration,Target Thickness,Spot Size,Focal Distance,Contrast,Max Proton Energy,Total Proton Energy,Avg Proton Energy,Max Exact Energy,Total Exact Energy,Avg Exact Energy,Laser Energy,Conversion Efficiency
0,2.728946e+18,40.0,0.766247,1.5,0.0,1.326224e-07,0.969093,3.542212e+08,0.142894,0.969093,3.542212e+08,0.142894,3.055517,0.018572
1,1.061515e+18,40.0,1.372547,1.5,0.0,6.800326e-07,0.083561,4.293747e+06,0.021613,0.083561,4.293747e+06,0.021613,1.059857,0.000649
2,3.545377e+18,40.0,2.546106,1.5,0.0,1.719615e-07,0.505331,1.803240e+08,0.097923,0.505331,1.803240e+08,0.097923,3.659088,0.007895
3,2.724723e+18,40.0,3.257884,1.5,0.0,1.512016e-07,0.446361,1.263015e+08,0.087432,0.446361,1.263015e+08,0.087432,2.997170,0.006751
4,2.632496e+18,40.0,3.421869,1.5,0.0,5.458767e-07,0.068507,5.749164e+06,0.018803,0.068507,5.749164e+06,0.018803,2.179157,0.000423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,6.501044e+18,40.0,4.708052,1.5,0.0,1.148893e-07,0.615096,3.996767e+08,0.119391,0.615096,3.996767e+08,0.119391,6.444476,0.009935
249996,3.440949e+18,40.0,2.633815,1.5,0.0,1.525120e-07,0.562106,2.070313e+08,0.105073,0.562106,2.070313e+08,0.105073,3.646153,0.009096
249997,1.473278e+18,40.0,2.683025,1.5,0.0,1.084751e-07,0.480875,8.680078e+07,0.086197,0.480875,8.680078e+07,0.086197,1.794923,0.007747
249998,1.519065e+18,40.0,2.855405,1.5,0.0,2.031555e-07,0.278398,3.987725e+07,0.058792,0.278398,3.987725e+07,0.058792,1.735060,0.003682


In [11]:
test_df.to_hdf('datasets/' + testing_tag + '.h5', key='df', mode='w')

In [12]:
test_df.sort_values(by='Max Proton Energy', ascending=True).head(10)

,Intensity,Pulse Duration,Target Thickness,Spot Size,Focal Distance,Contrast,Max Proton Energy,Total Proton Energy,Avg Proton Energy,Max Exact Energy,Total Exact Energy,Avg Exact Energy,Laser Energy,Conversion Efficiency
238718,5.924877e+18,40.0,1.629336,1.5,0.0,7.689128e-07,0.01,180517.947532,0.01,9.356587e-03,1.805179e+05,2.936644e-03,2.194953,1.317521e-05
123037,4.623534e+18,40.0,3.136604,1.5,0.0,9.201673e-07,0.01,96093.424081,0.01,7.006907e-03,9.609342e+04,2.212448e-03,1.875080,8.209870e-06
195025,9.850953e+18,40.0,4.742117,1.5,0.0,7.567984e-07,0.01,11.533136,0.01,8.236738e-05,1.153314e+01,2.731639e-05,0.779695,2.369655e-09
163680,8.581559e+18,40.0,3.748761,1.5,0.0,9.798025e-07,0.01,0.010000,0.01,3.537187e-19,0.000000e+00,0.000000e+00,0.000012,0.000000e+00
123052,7.219682e+18,40.0,4.669006,1.5,0.0,6.873046e-07,0.01,69571.987110,0.01,5.358236e-03,6.957199e+04,1.709476e-03,2.345852,4.751124e-06
78262,8.269153e+18,40.0,3.557841,1.5,0.0,7.631797e-07,0.01,2593.218533,0.01,1.094772e-03,2.593219e+03,3.579825e-04,1.536728,2.703365e-07
238939,9.304715e+18,40.0,2.887587,1.5,0.0,8.978485e-07,0.01,0.010000,0.01,1.449870e-10,3.953676e-12,4.947484e-11,0.009022,7.020150e-20
238942,6.624712e+18,40.0,2.979844,1.5,0.0,9.034657e-07,0.01,3808.086676,0.01,1.372916e-03,3.808087e+03,4.475280e-04,1.443779,4.225408e-07
78258,9.725965e+18,40.0,2.454394,1.5,0.0,9.578660e-07,0.01,0.010000,0.01,4.410625e-19,0.000000e+00,0.000000e+00,0.000014,0.000000e+00
102765,8.025105e+18,40.0,3.023844,1.5,0.0,7.070177e-07,0.01,21481.857926,0.01,3.051951e-03,2.148186e+04,9.849077e-04,2.001876,1.719085e-06
